In [55]:
# Load NeuroKit and other useful packages
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [35]:
df = pd.read_parquet('~/cloudfiles/code/users/arinjay.jain1/datasets/hackathon_dataset_decoded_mode.parquet')

In [36]:
df.head(5)

,Episode,Label,decoded_signal
0,DeviceOSEBCMPDZAZ7__EpisodeHFDMGC46COM2,0,"[70.0, -56.0, -97.0, -109.0, -55.0, 29.0, 59.0..."
1,Device/KDFTQ4HUMRD__EpisodeW5+1+HZDWVQE,1,"[8.0, 9.0, 9.0, 7.0, 7.0, 6.0, 13.0, 14.0, 10...."
2,DeviceDLFOL6DT/WOV__EpisodeUYPCWAWAZYGZ,0,"[-4.0, -2.0, -1.0, 0.0, 1.0, 3.0, 3.0, 1.0, 0...."
3,DeviceQEMR3QU3IMXF__EpisodeXF8CO+KDN0TN,0,"[6.0, 7.0, 8.0, 7.0, 5.0, 6.0, 6.0, 6.0, 6.0, ..."
4,DeviceXLBY/IIVTCUE__EpisodeUOW1CTCRGD5M,0,"[-4.0, -4.0, -3.0, -4.0, -4.0, -4.0, -4.0, -3...."


In [43]:
from grpc import local_server_credentials
import numpy as np
import pandas as pd
from scipy.fftpack import fft
from numpy import trapz
import base64
import pickle
import tempfile # Required import, do not remove
# import onnxruntime as nxrun
import neurokit2 as nk
from sklearn.metrics import f1_score
def align_fft(sigs_parsed,nbin=128):
        N=4*60*128
        s_new=[]

        for s in (sigs_parsed):
                if(len(s))>=N:
                        s_new.append(list(s[:N]))
                else:
                        z=N-len(s)
                        lst = [0] * z
                        c=np.hstack([s,lst])
                        s_new.append(c)      
        sr =  1/128
        T=N/sr
        n=np.arange(N)
        freq=n/T
        
        n_one_size=int(N/2)
        f_oneside=freq[:n_one_size]

        pe=int(N/nbin/2)
       
        for i, sig in enumerate(s_new):
            
            s_new[i]=np.abs(fft(sig)[:n_one_size])
            s_new[i]= s_new[i].reshape(-1,pe).mean(1)        
        #     xmax= max(s_new[i])
        #     xmin= min(s_new[i])
        #     s_new[i]=(s_new[i]-xmin)/(xmax-xmin)

            

        return s_new
def classify(signal: np.ndarray) -> int:
    assert type(signal) == np.ndarray, 'signal should be np.ndarray'
    peaks, _ = nk.ecg_peaks(signal, sampling_rate=120)
    hrv_time = nk.hrv_time(peaks, sampling_rate=120)
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=120, normalize=True)
    frame2=[hrv_time,hrv_freq]
    df_o=pd.concat(frame2,axis=1)
    t=nk.ecg_quality(signal, method="zhao2018",sampling_rate=128)
    if(t=='Unacceptable'):
        df_o['quality'] = 0
    else:
        df_o['quality'] = 1
    df_o.drop(columns=['HRV_SDANN2','HRV_SDNNI2','HRV_SDANN5','HRV_SDNNI5','HRV_ULF'],inplace=True)
    df_o= df_o.fillna(0)
    nk_feature=list(df_o.values[0])
    local=[]
    local.append(signal)
#     local=parse_signal(local)
    fft_feature=align_fft(local,nbin=256)    
    c=np.hstack([fft_feature[0],nk_feature])
    local=[]
    local.append(c)
    #res=model.predict(local)[0]
    return local

In [51]:
df['new_signal'] = df['decoded_signal'].apply(classify)

In [52]:
df

,Episode,Label,decoded_signal,new_signal
0,DeviceOSEBCMPDZAZ7__EpisodeHFDMGC46COM2,0,"[70.0, -56.0, -97.0, -109.0, -55.0, 29.0, 59.0...","[[1869.8386173835156, 1216.6577899963092, 969...."
1,Device/KDFTQ4HUMRD__EpisodeW5+1+HZDWVQE,1,"[8.0, 9.0, 9.0, 7.0, 7.0, 6.0, 13.0, 14.0, 10....","[[2214.8873601300456, 3127.858671714706, 2490...."
2,DeviceDLFOL6DT/WOV__EpisodeUYPCWAWAZYGZ,0,"[-4.0, -2.0, -1.0, 0.0, 1.0, 3.0, 3.0, 1.0, 0....","[[994.5445876290639, 1693.62505950252, 932.793..."
3,DeviceQEMR3QU3IMXF__EpisodeXF8CO+KDN0TN,0,"[6.0, 7.0, 8.0, 7.0, 5.0, 6.0, 6.0, 6.0, 6.0, ...","[[1212.161953349213, 1623.170260800203, 1284.2..."
4,DeviceXLBY/IIVTCUE__EpisodeUOW1CTCRGD5M,0,"[-4.0, -4.0, -3.0, -4.0, -4.0, -4.0, -4.0, -3....","[[848.4253403484178, 1717.0549162936684, 1705...."
...,...,...,...,...
27413,DeviceH09PHB8GSTGK__EpisodeRD60JRL5WKZB,1,"[7.0, 7.0, 6.0, 6.0, 7.0, 6.0, 6.0, 5.0, 5.0, ...","[[1194.6971494984148, 1322.2063441084285, 2425..."
27414,DeviceAMJQPZYJ3MZD__Episode4JTSSSQSUXS1,0,"[-1.0, -3.0, -5.0, -6.0, -6.0, -6.0, -6.0, -6....","[[1795.9657216900355, 1493.5492756369958, 656...."
27415,DeviceYWKHBNNHO2WN__EpisodeNQQ4OG12I+QW,1,"[-1.0, -2.0, -1.0, -1.0, 0.0, 1.0, 1.0, 5.0, 1...","[[1676.744596072007, 2069.499395834926, 1860.1..."
27416,Device7C0VTWXNEWBJ__EpisodeAY/S6NCLKPIK,0,"[-51.0, -77.0, -32.0, 15.0, 13.0, 2.0, 4.0, 8....","[[1386.9612108494666, 949.9086032967457, 972.9..."


In [53]:
df[['new_signal', 'Label']].to_parquet('~/cloudfiles/code/users/arinjay.jain1/datasets/hackathon_dataset_new_signal.parquet')

In [22]:
df = pd.read_parquet('~/cloudfiles/code/users/arinjay.jain1/datasets/hackathon_dataset_new_signal.parquet')

In [23]:
#df['new_signal']

for i in range(len(df)):
    df['new_signal'][i] = df['new_signal'][i][0]

print(df)

                                              new_signal  Label
0      [1869.8386173835156, 1216.6577899963092, 969.5...      0
1      [2214.8873601300456, 3127.858671714706, 2490.1...      1
2      [994.5445876290639, 1693.62505950252, 932.7939...      0
3      [1212.161953349213, 1623.170260800203, 1284.28...      0
4      [848.4253403484178, 1717.0549162936684, 1705.8...      0
...                                                  ...    ...
27413  [1194.6971494984148, 1322.2063441084285, 2425....      1
27414  [1795.9657216900355, 1493.5492756369958, 656.6...      0
27415  [1676.744596072007, 2069.499395834926, 1860.13...      1
27416  [1386.9612108494666, 949.9086032967457, 972.93...      0
27417  [1422.5390735063747, 2143.977269626268, 4026.1...      1

[27418 rows x 2 columns]


In [38]:
df['Label'].value_counts()

0    18388
1     9030
Name: Label, dtype: int64

In [39]:
df_lable_1 = df[df['Label'] == 1]
df= df.append([df_lable_1]*1,ignore_index=True)

In [40]:
df['Label'].value_counts()

0    18388
1    18060
Name: Label, dtype: int64

In [50]:
#CNN
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout, BatchNormalization

#splitting dataset to dependent and independent variable
X = df['new_signal'].values #independent values / features
y = df['Label'].values #dependent values / target

#splitting the datasets for training and testing process
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

X_train = np.vstack(X_train).astype(np.float)
y_train = np.vstack(y_train).astype(np.int)
X_test = np.vstack(X_test).astype(np.float)
y_test = np.vstack(y_test).astype(np.int)


#size for the sets
print('size of X_train:', X_train.shape)
print('size of X_test:', X_test.shape)
print('size of y_train:', y_train.shape)
print('size of y_test:', y_test.shape)



#Reshape train and test data to (n_samples, 19184, 1), where each sample is of size (19184, 1)
X_train = np.array(X_train).reshape(X_train.shape[0],X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)

size of X_train: (25513, 285)
size of X_test: (10935, 285)
size of y_train: (25513, 1)
size of y_test: (10935, 1)
X Train shape:  (25513, 285, 1)
X Test shape:  (10935, 285, 1)


In [52]:
# Create sequential model 
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

model = tf.keras.models.Sequential()

model.add(Conv1D(filters=32, kernel_size=(3,), strides=3, activation='relu', padding='same', input_shape=(285, 1)))
model.add(BatchNormalization())

#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv1D(64, kernel_size=(3,), strides=2, activation='relu', padding='same'))
model.add(BatchNormalization())

model.add(Conv1D(128, kernel_size=(3,), strides=2, activation='relu', padding='same'))
model.add(Dropout(0.5))


model.add(Conv1D(256, kernel_size=(3,), strides=2, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(512, kernel_size=(3,), strides=2, activation='relu', padding='same'))
model.add(BatchNormalization())

model.add(Conv1D(256, kernel_size=(3,), strides=2, activation='relu', padding='same'))
model.add(BatchNormalization())

#model.add(Dropout(0.5))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv1D(filters=128, kernel_size=(3,), strides=2, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

#model.add(Conv1D(filters=64, kernel_size=(3,), strides=2, activation='relu', padding='same'))
#model.add(Dropout(0.5))
#model.add(BatchNormalization())

#model.add(Conv1D(filters=32, kernel_size=(3,), strides=2, activation='relu', padding='same'))
#model.add(Dropout(0.5))
#model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

opt = Adam(lr=0.001)#Adam(lr=0.001)
model.compile(optimizer=opt, loss=categorical_crossentropy, metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.75, patience=5, min_lr=0.000001)


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_56 (Conv1D)           (None, 95, 32)            128       
_________________________________________________________________
batch_normalization_19 (Batc (None, 95, 32)            128       
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 48, 64)            6208      
_________________________________________________________________
batch_normalization_20 (Batc (None, 48, 64)            256       
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 24, 128)           24704     
_________________________________________________________________
dropout_21 (Dropout)         (None, 24, 128)           0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 12, 256)         

In [53]:
model_history = model.fit(X_train, y_train,  epochs=100, batch_size=15, validation_data = (X_test, y_test), callbacks=[reduce_lr])
#model_history = model.fit(X_train, y_train,  epochs=20, batch_size=20, validation_data = (X_test, y_test))

Epoch 1/100
1701/1701 [==============================] - 19s 11ms/step - loss: 5.8939e-08 - accuracy: 0.4975 - val_loss: 5.9370e-08 - val_accuracy: 0.4979 - lr: 0.0010
Epoch 2/100
1701/1701 [==============================] - 18s 11ms/step - loss: 5.8939e-08 - accuracy: 0.5014 - val_loss: 5.9370e-08 - val_accuracy: 0.4984 - lr: 0.0010
Epoch 3/100
1701/1701 [==============================] - 19s 11ms/step - loss: 5.8939e-08 - accuracy: 0.4989 - val_loss: 5.9370e-08 - val_accuracy: 0.5012 - lr: 0.0010
Epoch 4/100
1701/1701 [==============================] - 19s 11ms/step - loss: 5.8939e-08 - accuracy: 0.5010 - val_loss: 5.9370e-08 - val_accuracy: 0.5036 - lr: 0.0010
Epoch 5/100
1701/1701 [==============================] - 19s 11ms/step - loss: 5.8939e-08 - accuracy: 0.4992 - val_loss: 5.9370e-08 - val_accuracy: 0.5007 - lr: 0.0010
Epoch 6/100
1701/1701 [==

In [ ]:
model.save('model_cnn_285')

In [5]:
def get_signal_dwt(signal):
    #pqrst
    ecg_signal = signal
    # Extract R-peaks locations
    _, rpeaks = nk.ecg_peaks(ecg_signal, sampling_rate=128)
    
    # Delineate the ECG signal and visualizing all peaks of ECG complexes
    signal_dwt, waves_dwt = nk.ecg_delineate(ecg_signal, 
                                         rpeaks, 
                                         sampling_rate=128, 
                                         method="dwt", 
                                         show=False, 
                                         show_type='all')
    # print(type(signal_dwt))
    print(signal_dwt)
    # print(signal_dwt['ECG_P_Onsets'].std())
    signal_dwt_std = pd.DataFrame(signal_dwt['ECG_P_Onsets'].std())
    signal_dwt_std.append(signal_dwt['ECG_P_Offsets'].std())
    signal_dwt_std.append(signal_dwt['ECG_R_Onsets'].std())
    signal_dwt_std.append(signal_dwt['ECG_R_Offsets'].std())
    

    # Automatically process the (raw) ECG signal
    return signal_dwt_std

In [7]:
sigs_dwt = pd.DataFrame()
N = df.shape[0]
i = 0
print(N)
for sig in df['decoded_signal']:
    sig_parsed = get_signal_dwt(sig)
    sigs_dwt.append(sig_parsed)
    i = i + 1
    if i % 1000 == 0:
        print(round(100*(i + 1) / N, 2), ' percent complete.')
    if i == 5:
        break

sigs_dwt

27418
       ECG_P_Peaks  ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  \
0                0             0              0            0             0   
1                0             0              0            0             0   
2                0             0              0            0             0   
3                0             0              0            0             0   
4                0             0              0            0             0   
...            ...           ...            ...          ...           ...   
19179            0             0              0            0             0   
19180            0             0              0            0             0   
19181            0             0              0            0             0   
19182            0             0              0            0             0   
19183            0             0              0            0             0   

       ECG_R_Offsets  ECG_S_Peaks  ECG_T_Peaks  ECG_T_Ons

ValueError: DataFrame constructor not properly called!

In [20]:
def generategraphs(sig):
    #interval related analysis
    # Process ecg
    ecg_signals, info = nk.ecg_process(sig, sampling_rate=128)
    print(nk.ecg_intervalrelated(ecg_signals))
       
    # Find peaks
    # peaks, info = nk.ecg_peaks(df['signals_parsed'][i], sampling_rate=128)
    # hrv_indices = nk.hrv(peaks, sampling_rate=100, show=True)
    # hrv_indices
    
    # Automatically process the (raw) ECG signal
    return nk.ecg_intervalrelated(ecg_signals)


In [23]:
sigs_para = pd.DataFrame()
N = df.shape[0]
i = 0
print(N)
for sig in df['decoded_signal']:
    sig_parsed = generategraphs(sig)
    sigs_para.append(sig_parsed)

sigs_para

27418
   ECG_Rate_Mean  HRV_MeanNN   HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  \
0      69.625698  111.111111  23.088435         NaN         NaN         NaN   

   HRV_SDNNI2  HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  ...  HRV_SampEn  HRV_ShanEn  \
0         NaN         NaN         NaN  27.262126  ...    0.625251    5.335306   

   HRV_FuzzyEn  HRV_MSEn  HRV_CMSEn  HRV_RCMSEn    HRV_CD   HRV_HFD   HRV_KFD  \
0      1.08047  1.082662   1.247778    1.288692  0.859994  1.915093  3.832229   

    HRV_LZC  
0  0.824206  

[1 rows x 90 columns]
   ECG_Rate_Mean  HRV_MeanNN  HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  \
0      89.334189   86.808219  16.69944         NaN         NaN         NaN   

   HRV_SDNNI2  HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD  ...  HRV_SampEn  HRV_ShanEn  \
0         NaN         NaN         NaN  24.523533  ...    1.451809    5.719856   

   HRV_FuzzyEn  HRV_MSEn  HRV_CMSEn  HRV_RCMSEn    HRV_CD   HRV_HFD   HRV_KFD  \
0     1.658489  0.758159   1.180165    1.673383  1.701946

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyError: '[19184] not in index'

In [1]:
df['decoded_signal'].iloc[7190]

NameError: name 'df' is not defined

In [14]:
import base64
import numpy as np
import pandas as pd
import neurokit2 as nk
import heartpy as hp
import csv

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



In [ ]:
def classify(signal_o: np.ndarray) -> int:
    # commenting this assrt out, it always breaks
    assert type(signal_o) == np.ndarray, 'signal should be np.ndarray'
    # params
    padded_len = 200
    beat_pairs = 40


    df = pd.DataFrame({"Signal": [signal_o]})

 
    
    # create 'corrected' signal column with flipping function
    # functionalize quantile method
    # input should be the raw signal
    def flip_correction(input_signal):
        # first create baseline corrected and flip/baseline corrected
        base_curve = hp.remove_baseline_wander(input_signal, sample_rate= 128)
        base_flip = hp.flip_signal(input_signal, keep_range=True)
        flipped = hp.remove_baseline_wander(base_flip, sample_rate= 128)
    
        # current metrics full trace, consider using partial
        med_s = np.quantile(base_curve, .5)
        q95_s = np.quantile(base_curve, .999)
        q05_s = np.quantile(base_curve, .001)
    
        med_sf = np.quantile(flipped, .5)
        q95_sf = np.quantile(flipped, .999)
        q05_sf = np.quantile(flipped, .001)
    
        input_ratio = (q95_s-med_s)/(q95_s-q05_s) # percent pos peak from baseline (full range)
        flip_ratio = (q95_sf-med_sf)/(q95_sf-q05_sf)
        
        if flip_ratio >= input_ratio:
            input_signal = base_flip

        return input_signal

    df['Signal_flip'] = df['Signal'].apply(flip_correction)    


    # optional truncate signal for epoch analysis
    def signal_truncate(ecg_sig, max_len):
        if len(ecg_sig) >= max_len:
            ecg_sig = ecg_sig[0:max_len]
        else:
            ecg_sig=ecg_sig
        return ecg_sig
    
    df['Signal_flip'] = df['Signal_flip'].apply(signal_truncate, max_len = 10000)


    # creating baseline corrected feature as well (for peak RR stuff)
    df['corrected_baseline'] = df['Signal_flip'].apply(hp.remove_baseline_wander, sample_rate= 128)

    
    # alternative R peak detection using nk2 (trying to beat ~15 minutes)    
    def nk_peak_method(in_signal):
        _, test_rr = nk.ecg_peaks(in_signal, sampling_rate=128)
    
        output = test_rr['ECG_R_Peaks']
        return output
    
    df['NK_Peak'] = df['corrected_baseline'].apply(nk_peak_method)
    
        
    # generate peak list for stacking
    peak_buffer = 50 # buffer before first beat after second beat
    
    def truncated_beats(peaks):
        # if peaks are under 10 beats, probably a failure of detection
        # in that event just create index of 100's
        if len(peaks) < 10:
            peaks = np.array(range(100,10000,90 ))
    
        # eliminate last beat
        peaks = peaks[0:(peaks.shape[0]-1)]
    
        # eliminate any index less than buffer
        peaks = peaks[peaks > peak_buffer ]
    
        return peaks
    
    df['NK_Peak'] = df['NK_Peak'].apply(truncated_beats) 


    # standardize signal used for stack/image building
    def ecg_std(ecg_in):
        ecg_out = (ecg_in - ecg_in.mean()) / ecg_in.std()
        return ecg_out
    
    # three options: flipped signal, raw signal, flip_baseline
    df['signal_std'] = df['Signal_flip'].apply(ecg_std)
    
    
    
    # build all stacked arrays
    num_signals = df.shape[0]
    
    full_pairs=np.empty([num_signals,padded_len, beat_pairs,1])
    
    def paired(start_1, stop_2, ecg_sig):
        # for first trace get stacked
        double_beat = ecg_sig[int(start_1):int(stop_2)]
    
        #double_beat = double_beat.to_numpy()
        double_beat = np.pad(double_beat, (0, max(0,padded_len - double_beat.size)), 'constant')
        #double_beat = double_beat.reshape((-1, 1))
        
        return double_beat
    
    def total_pairs(ecg_sig , peaks):
        all_pairs=np.empty([padded_len, beat_pairs])

    
        for ii in range(0,beat_pairs):
            if (ii+1) < len(peaks):
                start_1 = peaks[ii] - peak_buffer
                stop_2 = peaks[ii+1] + peak_buffer
            elif (ii+1) < 2*len(peaks):
                start_1 = peaks[ii-len(peaks)] - peak_buffer
                stop_2 = peaks[ii+1-len(peaks)] + peak_buffer
            elif (ii+1) < 3*len(peaks):
                start_1 = peaks[ii-2*len(peaks)] - peak_buffer
                stop_2 = peaks[ii+1-2*len(peaks)] + peak_buffer
            else:
                start_1 = peaks[ii-3*len(peaks)] - peak_buffer
                stop_2 = peaks[ii+1-3*len(peaks)] + peak_buffer
    
            if (stop_2 - start_1) > padded_len:
                stop_2 = start_1 + padded_len
    
            double_beat = paired(start_1=start_1,stop_2=stop_2, ecg_sig=ecg_sig)
            all_pairs[:,ii] = double_beat
            #all_pairs = np.concatenate((all_pairs, double_beat), axis=1)
        
        return all_pairs
    
    for kp in range(num_signals):
        all_pairs = total_pairs(ecg_sig = df['signal_std'][kp],peaks = df['NK_Peak'][kp])
        all_pairs = all_pairs[np.newaxis,...]
        all_pairs = all_pairs[..., np.newaxis]
        
        full_pairs[kp,:,:,:] = all_pairs
        #if kp == 0:
        #    full_pairs = all_pairs   
        #else:
        #    full_pairs = np.concatenate((full_pairs,all_pairs),axis=0)
    x_test = full_pairs
  

In [17]:
 df = pd.DataFrame({"Signal": [df['decoded_signal']]})

KeyError: 'decoded_signal'

In [20]:
def flip_correction(input_signal):
    # first create baseline corrected and flip/baseline corrected
    base_curve = hp.remove_baseline_wander(input_signal, sample_rate= 128)
    print(base_curve)
    base_flip = hp.flip_signal(input_signal, keep_range=True)
    flipped = hp.remove_baseline_wander(base_flip, sample_rate= 128)

    # current metrics full trace, consider using partial
    med_s = np.quantile(base_curve, .5)
    q95_s = np.quantile(base_curve, .999)
    q05_s = np.quantile(base_curve, .001)

    med_sf = np.quantile(flipped, .5)
    q95_sf = np.quantile(flipped, .999)
    q05_sf = np.quantile(flipped, .001)

    input_ratio = (q95_s-med_s)/(q95_s-q05_s) # percent pos peak from baseline (full range)
    flip_ratio = (q95_sf-med_sf)/(q95_sf-q05_sf)
    
    if flip_ratio >= input_ratio:
        input_signal = base_flip

    return input_signal

df['Signal_flip'] = df['Signal'].apply(flip_correction)   

[array([ 1.1128546e+02, -8.0565338e+01, -1.4459399e+02, ...,
         2.3629427e-02,  4.7909617e-01,  1.9705868e+00], dtype=float32)
 array([ 75.319176 , -35.316803 , -73.752    , ...,  -0.6543741,
          3.4069438,  35.58847  ], dtype=float32)
 array([  83.08392 ,  -59.02338 , -107.696625, ...,  -10.049489,
         -11.021132,  -15.688009], dtype=float32)                ...
 array([ 112.100044 ,  -68.28644  , -133.14758  , ...,    3.080871 ,
           3.430785 ,    5.9378405], dtype=float32)
 array([  63.48329 , -142.36777 , -164.00609 , ...,    9.789428,
          10.873001,    9.760797], dtype=float32)
 array([ 107.608925  ,  -74.5631    , -135.32785   , ...,    0.77655315,
           0.75792027,    1.8422608 ], dtype=float32)                   ]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
df

,Signal
0,"0 [70.0, -56.0, -97.0, -109.0, -55.0, 2..."


In [ ]:

# test classification dataset
from sklearn.datasets import make_classification
# define dataset
X, y = make_classification(n_samples=, n_features=20, n_informative=15, n_redundant=5, random_state=3)
# summarize the dataset
print(X.shape, y.shape)

In [ ]:

# evaluate random forest algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=3)
# define the model
model = RandomForestClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))